# Test the bnk module with a PCB accelerometer

Here we test the amplitude and frequency calibration of the BnK device using a [PCB accelerometer](https://www.pcb.com/spec_sheet.asp?model=352C650) attached to a [PCB calibrated shaker](https://www.pcb.com/spec_sheet.asp?model=394C06&item_id=2198).

In [ ]:
import time

from bnk.bnk import  OpenWav, WavHeader, Instrument

# Gather some data from a PCB accelerometer

The accelerometer is screwed onto the top of the shaker, and is connected to channel 4 of the BnK device. 

In [ ]:
bnk_ip = "10.1.1.66"
ADAC = Instrument(bnk_ip)

In [ ]:
ADAC.disable_all()
ADAC.set_samplerate(16384)
ADAC.set_name("PCB calibrated shaker")
ADAC.set_channel(    
                channel=4,
                name='PCB 394C06 SN 2706 1 g at 159.2 Hz',
                c_filter='7.0 Hz',
                c_range='10 Vpeak',
                sensitivity=0.0991,
                unit='g',
                powered=True,
                serialNumber='LW207441',
                transducerType='PCB352C65',
                )
ADAC.powerup()
print("Waiting for amplifier in PCB to stabilise ...")
time.sleep(20)
print("Recording")
recording_id = ADAC.record(2)
ADAC.powerdown()

In [ ]:
ADAC.list_recordings(start = -5)

In [ ]:
WAV_file = ADAC.get_wav(directory='../samples', recording_id=recording_id)
print(WAV_file)
ADAC.delete_recording(recording_id=recording_id)

## Load and display the data

In [ ]:
WAV_file = '../samples/PCB_calibrated_shaker_20190805205924.wav'
wav_data, metadata, json_data = OpenWav(WAV_file)

In [ ]:
#!conda install -c conda-forge bokeh -y

In [ ]:
from ipywidgets import interact
import numpy as np

from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
output_notebook()

In [ ]:
plt = figure(
    title = metadata['Label'],
    width = 800,
    height = 400,
    tools = ['xwheel_zoom','xpan','save','reset']
    
    )

sr = metadata['SampleRate']
N = len(wav_data)
delta_t = 1.0/sr
t = np.arange(0,N*delta_t,delta_t)

phase = 4.4
amp = 1
freq = 159.2
n = 1000

c = 0 # Channel
plt.line(t[:n], 
         wav_data[:n,c], 
         line_color='blue',
         legend=metadata['ChannelNames'][c]
        )
r = plt.line(t[:n],
             amp*np.sin(2*np.pi*freq*t[:n] + phase),
             line_color='red', 
             legend=f"{amp}  sin({freq} * 2πt + {phase})",
            )

plt.legend.click_policy="hide"

def update(phase=0):
    r.data_source.data['y'] = amp*np.sin(2*np.pi*freq*t[:n] + phase)
    push_notebook()

show(plt, notebook_handle=True)

In [ ]:
interact(update, phase=(0, 2*np.pi, 0.1))